In [49]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import pandas_gbq
from pykrx import stock
from pykrx import bond
import FinanceDataReader as fdr


from time import sleep

import psycopg2 as pg2
from sqlalchemy import create_engine

from datetime import datetime
from datetime import timedelta

import os
import time

import glob
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


# 경로 변경
os.chdir('/home/shjj08choi/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-396200'
dataset_id = 'finance_mlops'

# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-1'     # 서비스 계정 생성한 bucket 이름 입력


In [ ]:


conn = st.experimental_connection('gcs', type=FilesConnection)
kor_stock_ohlcv = conn.read("finance-mlops-1/data_crawler/kor_stock_ohlcv/kor_stock_ohlcv_20230825.csv", 
                      input_format="csv", ttl=600)
                      
kor_ticker_list = conn.read("finance-mlops-1/data_crawler/kor_ticker_list/kor_ticker_list_20230825.csv", 
                      input_format="csv", ttl=600)

In [ ]:
kor_stock_ohlcv['MA120'] = kor_stock_ohlcv['close'].rolling(window=120).mean()
kor_stock_ohlcv['MA60'] = kor_stock_ohlcv['close'].rolling(window=60).mean()
kor_stock_ohlcv['MA20'] = kor_stock_ohlcv['close'].rolling(window=20).mean()
kor_stock_ohlcv['MA5'] = kor_stock_ohlcv['close'].rolling(window=5).mean()


kor_stock_ohlcv['ticker'] = kor_stock_ohlcv['ticker'].astype(str)

In [ ]:
# df1 = kor_stock_ohlcv[kor_stock_ohlcv['date'] == '2023-07-21']
df1 = pd.merge(kor_stock_ohlcv, kor_ticker_list, 
        on = 'ticker', 
        how = 'left')


In [45]:


now = datetime.now()
# now = now + timedelta(days=-2)
today_date1 = now.strftime('%Y%m%d')
today_date2 = now.strftime('%Y-%m-%d')
today_date_time_csv = now.strftime("%Y%m%d_%H%M")

In [50]:

def upload_df(data, file_name, project_id, dataset_id, today_date1):
    if not os.path.exists(f'data_crawler/streamlit_data/{file_name}'):
        os.makedirs(f'data_crawler/streamlit_data/{file_name}')

    try:
        if not os.path.exists(f'data_crawler/streamlit_data/{file_name}/{ticker_nm}_{today_date1}.csv'):
            data.to_csv(f'data_crawler/streamlit_data/{file_name}/{ticker_nm}_{today_date1}.csv', index=False, mode='w')
        else:
            data.to_csv(f'data_crawler/{file_name}/{ticker_nm}_{today_date1}.csv', index=False, mode='a', header=False)
        print(f'{ticker_nm}_로컬CSV저장_success')
    except:
        print(f'{ticker_nm}_로컬CSV저장_fail')


    # Google Storage 적재
    source_file_name = f'data_crawler/streamlit_data/{file_name}/{ticker_nm}_{today_date1}.csv'    # GCP에 업로드할 파일 절대경로
    destination_blob_name = f'data_crawler/streamlit_data/{file_name}/{ticker_nm}_{today_date1}.csv'    # 업로드할 파일을 GCP에 저장할 때의 이름
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)

In [51]:
file_name = 'kor_stock_ohlcv'

for ticker_nm in ticker_list:
    df_ticker = df1[df1['ticker'] == ticker_nm]
    
    upload_df(df_ticker, file_name, project_id, dataset_id, today_date1)

095570_로컬CSV저장_success
006840_로컬CSV저장_success
027410_로컬CSV저장_success
282330_로컬CSV저장_success
138930_로컬CSV저장_success
001460_로컬CSV저장_success
001465_로컬CSV저장_success
001040_로컬CSV저장_success
079160_로컬CSV저장_success
00104K_로컬CSV저장_success
000120_로컬CSV저장_success
011150_로컬CSV저장_success
011155_로컬CSV저장_success
001045_로컬CSV저장_success
097950_로컬CSV저장_success
097955_로컬CSV저장_success
000480_로컬CSV저장_success
000590_로컬CSV저장_success
012030_로컬CSV저장_success
016610_로컬CSV저장_success
005830_로컬CSV저장_success
000990_로컬CSV저장_success
139130_로컬CSV저장_success
001530_로컬CSV저장_success
000210_로컬CSV저장_success
001880_로컬CSV저장_success
000215_로컬CSV저장_success
375500_로컬CSV저장_success
37550L_로컬CSV저장_success
37550K_로컬CSV저장_success
007340_로컬CSV저장_success
004840_로컬CSV저장_success
155660_로컬CSV저장_success
069730_로컬CSV저장_success
017940_로컬CSV저장_success
365550_로컬CSV저장_success
383220_로컬CSV저장_success
007700_로컬CSV저장_success
114090_로컬CSV저장_success
078930_로컬CSV저장_success
006360_로컬CSV저장_success
001250_로컬CSV저장_success
007070_로컬CSV저장_success
078935_로컬CS

377190_로컬CSV저장_success
013570_로컬CSV저장_success
210540_로컬CSV저장_success
115390_로컬CSV저장_success
032350_로컬CSV저장_success
089860_로컬CSV저장_success
330590_로컬CSV저장_success
000400_로컬CSV저장_success
023530_로컬CSV저장_success
020150_로컬CSV저장_success
280360_로컬CSV저장_success
004000_로컬CSV저장_success
286940_로컬CSV저장_success
004990_로컬CSV저장_success
00499K_로컬CSV저장_success
005300_로컬CSV저장_success
005305_로컬CSV저장_success
011170_로컬CSV저장_success
071840_로컬CSV저장_success
027740_로컬CSV저장_success
357430_로컬CSV저장_success
001080_로컬CSV저장_success
088980_로컬CSV저장_success
094800_로컬CSV저장_success
138040_로컬CSV저장_success
090370_로컬CSV저장_success
017180_로컬CSV저장_success
009900_로컬CSV저장_success
012690_로컬CSV저장_success
005360_로컬CSV저장_success
204210_로컬CSV저장_success
009680_로컬CSV저장_success
009580_로컬CSV저장_success
009200_로컬CSV저장_success
033920_로컬CSV저장_success
008420_로컬CSV저장_success
025560_로컬CSV저장_success
007120_로컬CSV저장_success
396690_로컬CSV저장_success
357250_로컬CSV저장_success
085620_로컬CSV저장_success
006800_로컬CSV저장_success
00680K_로컬CSV저장_success
006805_로컬CS

002785_로컬CSV저장_success
009310_로컬CSV저장_success
000650_로컬CSV저장_success
012600_로컬CSV저장_success
033250_로컬CSV저장_success
035720_로컬CSV저장_success
323410_로컬CSV저장_success
377300_로컬CSV저장_success
006380_로컬CSV저장_success
109070_로컬CSV저장_success
001620_로컬CSV저장_success
029460_로컬CSV저장_success
281820_로컬CSV저장_success
381970_로컬CSV저장_success
145270_로컬CSV저장_success
417310_로컬CSV저장_success
357120_로컬CSV저장_success
007815_로컬CSV저장_success
007810_로컬CSV저장_success
00781K_로컬CSV저장_success
003690_로컬CSV저장_success
192820_로컬CSV저장_success
044820_로컬CSV저장_success
005070_로컬CSV저장_success
005420_로컬CSV저장_success
071950_로컬CSV저장_success
002020_로컬CSV저장_success
003070_로컬CSV저장_success
003075_로컬CSV저장_success
450140_로컬CSV저장_success
45014K_로컬CSV저장_success
002025_로컬CSV저장_success
120110_로컬CSV저장_success
120115_로컬CSV저장_success
138490_로컬CSV저장_success
021240_로컬CSV저장_success
036420_로컬CSV저장_success
031820_로컬CSV저장_success
192400_로컬CSV저장_success
284740_로컬CSV저장_success
264900_로컬CSV저장_success
26490K_로컬CSV저장_success
005740_로컬CSV저장_success
005745_로컬CS

011040_로컬CSV저장_success
024910_로컬CSV저장_success
198440_로컬CSV저장_success
049720_로컬CSV저장_success
014570_로컬CSV저장_success
348150_로컬CSV저장_success
098460_로컬CSV저장_success
035290_로컬CSV저장_success
900280_로컬CSV저장_success
215000_로컬CSV저장_success
121440_로컬CSV저장_success
366030_로컬CSV저장_success
014200_로컬CSV저장_success
029480_로컬CSV저장_success
026910_로컬CSV저장_success
397880_로컬CSV저장_success
421800_로컬CSV저장_success
440790_로컬CSV저장_success
456490_로컬CSV저장_success
053270_로컬CSV저장_success
066620_로컬CSV저장_success
043650_로컬CSV저장_success
006050_로컬CSV저장_success
060480_로컬CSV저장_success
078130_로컬CSV저장_success
307750_로컬CSV저장_success
035080_로컬CSV저장_success
204020_로컬CSV저장_success
114450_로컬CSV저장_success
186230_로컬CSV저장_success
900070_로컬CSV저장_success
204620_로컬CSV저장_success
019660_로컬CSV저장_success
053260_로컬CSV저장_success
282720_로컬CSV저장_success
036190_로컬CSV저장_success
049080_로컬CSV저장_success
420770_로컬CSV저장_success
035460_로컬CSV저장_success
308100_로컬CSV저장_success
407400_로컬CSV저장_success
187790_로컬CSV저장_success
286750_로컬CSV저장_success
121600_로컬CS

226340_로컬CSV저장_success
014470_로컬CSV저장_success
008470_로컬CSV저장_success
338220_로컬CSV저장_success
100120_로컬CSV저장_success
406820_로컬CSV저장_success
337930_로컬CSV저장_success
099390_로컬CSV저장_success
064480_로컬CSV저장_success
288330_로컬CSV저장_success
365900_로컬CSV저장_success
251630_로컬CSV저장_success
018290_로컬CSV저장_success
044480_로컬CSV저장_success
033560_로컬CSV저장_success
369370_로컬CSV저장_success
126340_로컬CSV저장_success
121800_로컬CSV저장_success
148140_로컬CSV저장_success
082800_로컬CSV저장_success
318410_로컬CSV저장_success
419540_로컬CSV저장_success
146320_로컬CSV저장_success
200780_로컬CSV저장_success
141000_로컬CSV저장_success
083650_로컬CSV저장_success
445360_로컬CSV저장_success
142760_로컬CSV저장_success
065170_로컬CSV저장_success
086670_로컬CSV저장_success
335890_로컬CSV저장_success
138580_로컬CSV저장_success
082920_로컬CSV저장_success
054220_로컬CSV저장_success
042370_로컬CSV저장_success
050090_로컬CSV저장_success
307870_로컬CSV저장_success
357880_로컬CSV저장_success
032850_로컬CSV저장_success
148780_로컬CSV저장_success
238200_로컬CSV저장_success
093190_로컬CSV저장_success
065450_로컬CSV저장_success
210120_로컬CS

109610_로컬CSV저장_success
031860_로컬CSV저장_success
306040_로컬CSV저장_success
457940_로컬CSV저장_success
435870_로컬CSV저장_success
455910_로컬CSV저장_success
096630_로컬CSV저장_success
069510_로컬CSV저장_success
041910_로컬CSV저장_success
234300_로컬CSV저장_success
039440_로컬CSV저장_success
098660_로컬CSV저장_success
052020_로컬CSV저장_success
237690_로컬CSV저장_success
050760_로컬CSV저장_success
288620_로컬CSV저장_success
317830_로컬CSV저장_success
058610_로컬CSV저장_success
043340_로컬CSV저장_success
187660_로컬CSV저장_success
054630_로컬CSV저장_success
200710_로컬CSV저장_success
096690_로컬CSV저장_success
298380_로컬CSV저장_success
203400_로컬CSV저장_success
195990_로컬CSV저장_success
003800_로컬CSV저장_success
088800_로컬CSV저장_success
241840_로컬CSV저장_success
312610_로컬CSV저장_success
172670_로컬CSV저장_success
072990_로컬CSV저장_success
227100_로컬CSV저장_success
239610_로컬CSV저장_success
353060_로컬CSV저장_success
148930_로컬CSV저장_success
044780_로컬CSV저장_success
357230_로컬CSV저장_success
071670_로컬CSV저장_success
045660_로컬CSV저장_success
224110_로컬CSV저장_success
230980_로컬CSV저장_success
021080_로컬CSV저장_success
089530_로컬CS

101930_로컬CSV저장_success
333430_로컬CSV저장_success
068330_로컬CSV저장_success
178780_로컬CSV저장_success
019540_로컬CSV저장_success
094820_로컬CSV저장_success
950140_로컬CSV저장_success
049550_로컬CSV저장_success
389020_로컬CSV저장_success
254120_로컬CSV저장_success
043910_로컬CSV저장_success
234920_로컬CSV저장_success
289220_로컬CSV저장_success
174880_로컬CSV저장_success
049630_로컬CSV저장_success
417840_로컬CSV저장_success
110020_로컬CSV저장_success
208140_로컬CSV저장_success
040420_로컬CSV저장_success
045510_로컬CSV저장_success
217190_로컬CSV저장_success
095700_로컬CSV저장_success
072520_로컬CSV저장_success
122310_로컬CSV저장_success
361390_로컬CSV저장_success
187420_로컬CSV저장_success
225220_로컬CSV저장_success
123330_로컬CSV저장_success
159580_로컬CSV저장_success
147830_로컬CSV저장_success
033100_로컬CSV저장_success
079370_로컬CSV저장_success
054950_로컬CSV저장_success
023440_로컬CSV저장_success
090470_로컬CSV저장_success
287410_로컬CSV저장_success
137950_로컬CSV저장_success
033320_로컬CSV저장_success
417500_로컬CSV저장_success
204270_로컬CSV저장_success
026040_로컬CSV저장_success
126880_로컬CSV저장_success
322510_로컬CSV저장_success
033050_로컬CS

163730_로컬CSV저장_success
417180_로컬CSV저장_success
446750_로컬CSV저장_success
448370_로컬CSV저장_success
454750_로컬CSV저장_success
454640_로컬CSV저장_success
400560_로컬CSV저장_success
406760_로컬CSV저장_success
418170_로컬CSV저장_success
427950_로컬CSV저장_success
430230_로컬CSV저장_success
435620_로컬CSV저장_success
299030_로컬CSV저장_success
067310_로컬CSV저장_success
372290_로컬CSV저장_success
166090_로컬CSV저장_success
136480_로컬CSV저장_success
003380_로컬CSV저장_success
101670_로컬CSV저장_success
365590_로컬CSV저장_success
149980_로컬CSV저장_success
013030_로컬CSV저장_success
126700_로컬CSV저장_success
106080_로컬CSV저장_success
377400_로컬CSV저장_success
400840_로컬CSV저장_success
450050_로컬CSV저장_success
221840_로컬CSV저장_success
106190_로컬CSV저장_success
373200_로컬CSV저장_success
066130_로컬CSV저장_success
004590_로컬CSV저장_success
039340_로컬CSV저장_success
034950_로컬CSV저장_success
222980_로컬CSV저장_success
256840_로컬CSV저장_success
025550_로컬CSV저장_success
017890_로컬CSV저장_success
080720_로컬CSV저장_success
063570_로컬CSV저장_success
041460_로컬CSV저장_success
101680_로컬CSV저장_success
039740_로컬CSV저장_success
053300_로컬CS

In [37]:
ticker_list = kor_ticker_list['ticker'].unique()

1394
1394
1394
1394
1394
1394
1394
1394
1394
1000
1394
1394
1394
1394
1394



KeyboardInterrupt



,date,open,high,low,close,volume,price_change_percentage,ticker,MA120,MA60,MA20,MA5,corp_name,market
0,2018-01-02,6980,6990,6850,6970,60294,0.000000,095570,NaN,NaN,NaN,NaN,AJ네트웍스,KOSPI
1,2018-01-03,6970,7000,6910,6920,45848,-0.717360,095570,NaN,NaN,NaN,NaN,AJ네트웍스,KOSPI
2,2018-01-04,6990,7100,6920,7070,53176,2.167630,095570,NaN,NaN,NaN,NaN,AJ네트웍스,KOSPI
3,2018-01-05,7080,7300,7040,7250,58573,2.545969,095570,NaN,NaN,NaN,NaN,AJ네트웍스,KOSPI
4,2018-01-08,7330,7330,7100,7130,47104,-1.655172,095570,NaN,NaN,NaN,7068.0,AJ네트웍스,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3198931,2023-08-21,6930,6930,6540,6600,35728,0.000000,238490,6885.750000,7241.666667,6513.5,6678.0,힘스,KOSDAQ
3198932,2023-08-22,6600,6700,6500,6600,13143,0.000000,238490,6888.416667,7246.166667,6502.0,6668.0,힘스,KOSDAQ
3198933,2023-08-23,6680,6690,6540,6560,16390,-0.606061,238490,6890.000000,7250.000000,6503.5,6606.0,힘스,KOSDAQ
3198934,2023-08-24,6600,6750,6570,6750,12363,2.896341,238490,6893.750000,7225.500000,6526.0,6622.0,힘스,KOSDAQ


In [26]:
kor_ticker_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2620 entries, 0 to 2619
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ticker     2620 non-null   object
 1   corp_name  2620 non-null   object
 2   market     2620 non-null   object
dtypes: object(3)
memory usage: 61.5+ KB


In [31]:
kor_stock_ohlcv[kor_stock_ohlcv['ticker'] == 238490]

,date,open,high,low,close,volume,price_change_percentage,ticker,MA120,MA60,MA20,MA5
3197542,2018-01-02,11971,11971,11470,11796,64519,-1.675419,238490,8697.716667,7995.100000,7180.30,7709.2
3197543,2018-01-03,11796,11896,11721,11821,43605,0.211936,238490,8711.725000,8038.450000,7407.85,8759.4
3197544,2018-01-04,11871,11996,11721,11721,44689,-0.845952,238490,8724.400000,8079.966667,7630.40,9749.6
3197545,2018-01-05,11570,11721,11245,11471,74859,-2.132924,238490,8736.908333,8117.483333,7843.95,10689.8
3197546,2018-01-08,11370,11445,10844,10870,103475,-5.239299,238490,8744.325000,8147.483333,8033.45,11535.8
...,...,...,...,...,...,...,...,...,...,...,...,...
3198931,2023-08-21,6930,6930,6540,6600,35728,0.000000,238490,6885.750000,7241.666667,6513.50,6678.0
3198932,2023-08-22,6600,6700,6500,6600,13143,0.000000,238490,6888.416667,7246.166667,6502.00,6668.0
3198933,2023-08-23,6680,6690,6540,6560,16390,-0.606061,238490,6890.000000,7250.000000,6503.50,6606.0
3198934,2023-08-24,6600,6750,6570,6750,12363,2.896341,238490,6893.750000,7225.500000,6526.00,6622.0


In [25]:
kor_ticker_list[kor_ticker_list['ticker'] == '238490']

,ticker,corp_name,market
2619,238490,힘스,KOSDAQ
